# 数据预处理（json2xlsx）

In [2]:
import os
import json
from datetime import datetime


def merge_json_files(keyword, dir_path, output_path_prefix):
    """
    合并目录下某关键词相关的 JSON 文件，分别合并主 JSON、评论和用户信息。
    
    :param keyword: 关键词，例如 'liziqi' 或 'black-myth'
    :param dir_path: 输入目录路径
    :param output_path_prefix: 输出文件路径前缀，例如 './new_tweets/liziqi_merged'
    """
    # 筛选出与关键词相关的文件
    keyword_files = [f for f in os.listdir(dir_path) if f.startswith(keyword)]
    main_files = sorted([f for f in keyword_files if "_comments" not in f and "_user_info" not in f])
    comment_files = sorted([f for f in keyword_files if "_comments" in f])
    user_info_files = sorted([f for f in keyword_files if "_user_info" in f])

    # 合并 JSON 数据
    merged_main_data = []  # 用列表存储主数据
    merged_comment_data = {}
    merged_user_info_data = []

    # 合并主文件
    for file in main_files:
        file_path = os.path.join(dir_path, file)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            # 将所有 Tweet 数据合并到列表中
            merged_main_data.extend(data.values())

    # 合并评论文件
    for file in comment_files:
        file_path = os.path.join(dir_path, file)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            merged_comment_data.update(data)

    # 合并用户信息文件
    for file in user_info_files:
        file_path = os.path.join(dir_path, file)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            merged_user_info_data.extend(data)

    # 推断起始和终止日期
    def extract_date_range(files):
        dates = []
        for filename in files:
            parts = filename.split("_")
            if len(parts) >= 3:
                try:
                    dates.append(parts[1])
                    dates.append(parts[2].split(".")[0])
                except IndexError:
                    pass
        dates = sorted(set(datetime.strptime(d, "%Y-%m-%d") for d in dates))
        return dates[0].strftime("%Y-%m-%d"), dates[-1].strftime("%Y-%m-%d")

    start_date, end_date = extract_date_range(main_files)

    # 输出文件路径
    main_output_path = f"{output_path_prefix}_{start_date}_{end_date}.json"
    comments_output_path = f"{output_path_prefix}_{start_date}_{end_date}_comments.json"
    user_info_output_path = f"{output_path_prefix}_{start_date}_{end_date}_user_info.json"

    # 写入文件
    with open(main_output_path, "w", encoding="utf-8") as f:
        json.dump(merged_main_data, f, ensure_ascii=False, indent=4)
    print(f"Main data saved to {main_output_path}")

    with open(comments_output_path, "w", encoding="utf-8") as f:
        json.dump(merged_comment_data, f, ensure_ascii=False, indent=4)
    print(f"Comments saved to {comments_output_path}")

    with open(user_info_output_path, "w", encoding="utf-8") as f:
        json.dump(merged_user_info_data, f, ensure_ascii=False, indent=4)
    print(f"User info saved to {user_info_output_path}")


# 使用示例
dir_path = "./new_tweets"
# merge_json_files("liziqi", dir_path, "./new_tweets/liziqi")
merge_json_files("black-myth", dir_path, "./new_tweets/black-myth")


Main data saved to ./new_tweets/black-myth_2024-08-20_2024-09-30.json
Comments saved to ./new_tweets/black-myth_2024-08-20_2024-09-30_comments.json
User info saved to ./new_tweets/black-myth_2024-08-20_2024-09-30_user_info.json


In [25]:
import os
import json
import pandas as pd


def generate_excel_from_json(main_json_path, comments_json_path, user_json_path, output_excel_path):
    """
    根据主 JSON、评论 JSON 和用户 JSON 生成 Excel 表格。
    """
    # 加载数据
    main_df = json_to_dataframe(main_json_path)
    comments_df = json_to_dataframe(comments_json_path, additional_columns={"source_tweet_id": None, "source_tweet_text": None})
    user_df = json_to_dataframe(user_json_path)

    # 调试列名
    print("Main DataFrame Columns:", main_df.columns)
    print("Comments DataFrame Columns:", comments_df.columns)
    print("User DataFrame Columns:", user_df.columns)

    # 修正 comments_df 的用户主键
    if "author_info.user_id" in comments_df.columns:
        comments_df.rename(columns={"author_info.user_id": "author"}, inplace=True)

    # 添加源推特信息到评论数据
    for idx, row in comments_df.iterrows():
        source_tweet_id = row.get("source_tweet_id", None)
        if not source_tweet_id:
            continue
        source_tweet = main_df[main_df["url"].str.contains(source_tweet_id, na=False)]
        if not source_tweet.empty:
            comments_df.at[idx, "source_tweet_text"] = source_tweet.iloc[0]["text"]

    # 合并用户信息到主数据
    main_df = merge_user_info(main_df, user_df, user_key="author")

    # 合并用户信息到评论数据
    comments_df = merge_user_info(comments_df, user_df, user_key="author")

    # 重排序评论数据列
    comment_columns = ["content", "author"] + [col for col in comments_df.columns if col not in ["content", "author"]]
    comments_df = comments_df[comment_columns]

    # 输出到 Excel
    with pd.ExcelWriter(output_excel_path) as writer:
        main_df.to_excel(writer, sheet_name="Main Data", index=False)
        comments_df.to_excel(writer, sheet_name="Comments", index=False)
        user_df.to_excel(writer, sheet_name="Users", index=False)
    print(f"Excel file saved to {output_excel_path}")


def json_to_dataframe(json_path, additional_columns=None):
    """
    加载 JSON 文件并转换为 DataFrame。
    :param json_path: JSON 文件路径
    :param additional_columns: 需要添加的额外列，字典形式
    :return: DataFrame
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    if isinstance(data, list):
        df = pd.DataFrame(data)
    elif isinstance(data, dict):
        # 展平评论数据
        data_lists = [value if len(value) > 0 else None for value in data.values()]
        data_lists = [value for value in data_lists if value is not None]
        all_comments = [comment for sublist in data_lists for comment in sublist]
        df = pd.json_normalize(all_comments)
    else:
        raise ValueError("Invalid JSON structure.")
    
    if additional_columns:
        for col, default_value in additional_columns.items():
            df[col] = default_value
    
    return df


def merge_user_info(df, user_df, user_key="author"):
    """
    将用户信息合并到主数据或评论数据中。
    :param df: 数据 DataFrame
    :param user_df: 用户信息 DataFrame
    :param user_key: 用户 ID 的列名
    :return: 合并后的 DataFrame
    """
    if user_key not in df.columns:
        raise KeyError(f"Key '{user_key}' not found in DataFrame columns: {df.columns.tolist()}")
    if "user_id" not in user_df.columns:
        raise KeyError(f"Key 'user_id' not found in User DataFrame columns: {user_df.columns.tolist()}")

    return df.merge(
        user_df,
        how="left",
        left_on=user_key,
        right_on="user_id",  # 假设用户信息的唯一标识列名为 user_id
        suffixes=("", "_user"),
    )


# 使用示例
# main_json_path = "./new_tweets/liziqi_2024-11-01_2024-11-16.json"
# comments_json_path = "./new_tweets/liziqi_2024-11-01_2024-11-16_comments.json"
# user_json_path = "./new_tweets/liziqi_2024-11-01_2024-11-16_user_info.json"
# output_excel_path = "./new_tweets/liziqi_2024-11-01_2024-11-16.xlsx"

# black-myth
main_json_path = "./new_tweets/black-myth_2024-08-20_2024-09-30.json"
comments_json_path = "./new_tweets/black-myth_2024-08-20_2024-09-30_comments.json"
user_json_path = "./new_tweets/black-myth_2024-08-20_2024-09-30_user_info.json"
output_excel_path = "./new_tweets/black-myth_2024-08-20_2024-09-30.xlsx"

generate_excel_from_json(main_json_path, comments_json_path, user_json_path, output_excel_path)


Main DataFrame Columns: Index(['text', 'likes', 'replies', 'retweets', 'reads', 'publish_time', 'url',
       'author'],
      dtype='object')
Comments DataFrame Columns: Index(['_id', 'pub_time', 'source_url', 'pictures', 'video_url',
       'play_duration', 'up_count', 'cmt_count', 'read_count', 'rtt_count',
       'share_count', 'collect_count', 'quote_count', 'play_count',
       'is_retweet', 'is_quote', 'embed_url', 'language', 'content',
       'ref_article_id', 'ref_source_url', 'root_article_id',
       'root_source_url', 'author_info.user_id', 'author_info.user_name',
       'author_info.user_url', 'retweet_info._id',
       'retweet_info.retweet_author.user_id',
       'retweet_info.retweet_author.user_name', 'retweet_info.url',
       'quote_info._id', 'quote_info.quote_author.user_id',
       'quote_info.quote_author.user_name', 'quote_info.url',
       'ref_author_info.user_id', 'ref_author_info.user_name',
       'ref_author_info.user_url', 'root_author_info.user_id',
  